In [1]:
# todo: сгенерировать свой трейн,Посмотреть на отклонения с высоким iou и с низким, 
# определить расхождение у разных пользователей для каждого айтема в тесте и трейне и подправить трейн под расхождение из теста
# обучать по батчам для айтемов. транспонировать айтемы и сделать фичи из них
# линейную регрессию проверить, использовать иоу в предсказании
# снизить тест сайз на трейн тест сплите
# вщять полностью икс мин и икс мах из фрема по иоу

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
np.random.seed(0)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import DBSCAN

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test_data.csv', 'sample_submit_zero.csv', 'train_answers.csv', 'train_data.csv']


In [2]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    try:
        iou = interArea / float(boxAArea + boxBArea - interArea)
    except ZeroDivisionError:
        iou = 1
    return iou

votes = pd.read_csv("../input/train_data.csv")
answers = pd.read_csv("../input/train_answers.csv")
test = pd.read_csv("../input/test_data.csv")
data = pd.concat([votes, test], axis=0)
X = data[['Xmin','Ymin', 'Xmax', 'Ymax']].values
clustering = DBSCAN(eps=3, min_samples=2).fit(X)
train =  pd.merge(votes, answers, how = 'left', left_on  = ['itemId'], right_on = ['itemId'])
train["iou"] = train[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true', 'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)
train.to_csv("train_iou.csv",index=False)

# # 
# train = train[0:200]
# # 

from shapely.geometry import box
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(train[['Xmin','Ymin', 'Xmax', 'Ymax']],test[['Xmin','Ymin', 'Xmax', 'Ymax']])
from sklearn.preprocessing import MinMaxScaler
scaler2 = MinMaxScaler()
scaler2.fit(train[['Xmin','Ymin', 'Xmax', 'Ymax']],test[['Xmin','Ymin', 'Xmax', 'Ymax']])

def make_feature(df):
    df2 = df.copy()
    dfmean = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()
    dfmin = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].min().reset_index()
    dfmax = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].max().reset_index()
    dfstd = df2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].std().reset_index()
    df =  pd.merge(df, dfmean, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_mean"))
    df =  pd.merge(df, dfmin, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_min"))
    df =  pd.merge(df, dfmax, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_max"))
    df =  pd.merge(df, dfstd, how = 'left',
                  left_on  = ['itemId'],
                  right_on = ['itemId'], suffixes=("","_std"))

    dfmean = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()
    dfmin = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].min().reset_index()
    dfmax = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].max().reset_index()
    dfstd = df2.groupby("userId")[['Xmin','Ymin', 'Xmax', 'Ymax']].std().reset_index()
    df =  pd.merge(df, dfmean, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_mean_us"))
    df =  pd.merge(df, dfmin, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_min_us"))
    df =  pd.merge(df, dfmax, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_max_us"))
    df =  pd.merge(df, dfstd, how = 'left',
                  left_on  = ['userId'],
                  right_on = ['userId'], suffixes=("","_std_us"))
    
    df['d1'] = df['Xmin_mean']-df['Xmin']
    df['d2'] = df['Ymin_mean']-df['Ymin']
    df['d3'] = df['Xmax_mean']-df['Xmax']
    df['d4'] = df['Ymax_mean']-df['Ymax']
    df['d5'] = df['Xmin_std']-df['Xmin']
    df['d6'] = df['Ymin_std']-df['Ymin']
    df['d7'] = df['Xmax_std']-df['Xmax']
    df['d8'] = df['Ymax_std']-df['Ymax']
    
    df['d9'] = df['Xmin']-df['d1']
    df['d10'] = df['Ymin']-df['d2']
    df['d11'] = df['Xmax']-df['d3']
    df['d12'] = df['Ymax']-df['d4']
    df['d13'] = df['Xmin']-df['d5']
    df['d14'] = df['Ymin']-df['d6']
    df['d15'] = df['Xmax']-df['d7']
    df['d16'] = df['Ymax']-df['d8']
    
    for el in ['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_mean','Ymin_mean', 'Xmax_mean', 'Ymax_mean', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'd10', 'd11', 'd12', 'd13', 'd14', 'd15', 'd16']:
        df[el+'_log10'] = np.log10(df[el])
    
    df['sum'] = df['Xmin']+df['Ymin']+df['Xmax']+df['Ymax']
    df['mult'] = df['Xmin']*df['Ymin']*df['Xmax']*df['Ymax']/100000000
    df['dif'] = df['Xmin']-df['Ymin']-df['Xmax']-df['Ymax']
    df['dif2'] = df['Xmin']-df['Ymin']+df['Xmax']-df['Ymax']
    df['dif3'] = df['Xmin']+df['Ymin']-df['Xmax']+df['Ymax']
    df['a'] = df['Xmax']-df['Xmin']
    df['b'] = df['Ymax']-df['Ymin']
    df['a_mean'] = df['Xmax_mean']-df['Xmin_mean']
    df['b_mean'] = df['Ymax_mean']-df['Ymin_mean']
    df['a_max'] = df['Xmax_max']-df['Xmin_max']
    df['b_max'] = df['Ymax_max']-df['Ymin_max']
    df['a_min'] = df['Xmax_min']-df['Xmin_min']
    df['b_min'] = df['Ymax_min']-df['Ymin_min']
    df['dd1'] = df['a_mean']/2 - df['a']/2
    df['dd2'] = df['b_mean']/2 - df['b']/2
    df['dd3'] = df['a_max']/2 - df['a']/2
    df['dd4'] = df['b_max']/2 - df['b']/2
    df['dd5'] = df['a_min']/2 - df['a']/2
    df['dd6'] = df['b_min']/2 - df['b']/2
    df['aa'] = df['Xmax']-df['Ymin']
    df['bb'] = df['Ymax']-df['Xmin']
    df['ab'] = df['b']>df['a']
    df['S'] = (df['Xmax']-df['Xmin'])*(df['Ymax']-df['Ymin'])
    df['Sob'] = (df['Xmax_max']-df['Xmin_min'])*(df['Ymax_max']-df['Ymin_min'])
    df['P'] = 2*((df['Xmax']-df['Xmin'])+(df['Ymax']-df['Ymin']))
    df['Pob'] = 2*((df['Xmax_max']-df['Xmin_min'])+(df['Ymax_max']-df['Ymin_min']))
    df['dist_from_0'] = ( (df['a']/2)**2 + (df['b']/2)**2 )**(1/2)
    
    xmin = []
    ymin = []
    xmax = []
    ymax = []
    for index, row in df.iterrows():
        itemid = row.itemId
#         print (itemid)
        e = list(df[df.itemId==itemid][['Xmin','Ymin', 'Xmax', 'Ymax']].values[0])
        rect = box(e[0],e[1],e[2],e[3])
        for index2, row2 in df[df.itemId==itemid].iterrows():
            e = []
            e.append(row2.Xmin)
            e.append(row2.Ymin)
            e.append(row2.Xmax)
            e.append(row2.Ymax)
#             print (e)
            rect = rect.intersection(box(e[0],e[1],e[2],e[3]))
#         print (rect.bounds)
        if rect.bounds ==():
            xmin.append(0)
            ymin.append(0)
            xmax.append(0)
            ymax.append(0)
        else:
            xmin.append(rect.bounds[0])
            ymin.append(rect.bounds[1])
            xmax.append(rect.bounds[2])
            ymax.append(rect.bounds[3])
    
    coords1 = pd.DataFrame(xmin, columns=['Xmin_intersec'])
    df = pd.concat([df, coords1], axis=1)
    coords2 = pd.DataFrame(ymin, columns=['Ymin_intersec'])
    df = pd.concat([df, coords2], axis=1)
    coords3 = pd.DataFrame(xmax, columns=['Xmax_intersec'])
    df = pd.concat([df, coords3], axis=1)
    coords4 = pd.DataFrame(ymax, columns=['Ymax_intersec'])
    df = pd.concat([df, coords4], axis=1)
    
    df['S_intersec'] = (df['Xmax_intersec']-df['Xmin_intersec'])*(df['Ymax_intersec']-df['Ymin_intersec'])
    df['P_intersec'] = 2*((df['Xmax_intersec']-df['Xmin_intersec'])+(df['Ymax_intersec']-df['Ymin_intersec']))
    df['Sob/S_intersec'] = df['Sob']/df['S_intersec']
    df['Pob/P_intersec'] = df['Pob']/df['P_intersec']
    df['S_intersec/Sob'] = df['S_intersec']/df['Sob']
    df['P_intersec/Pob'] = df['P_intersec']/df['Pob']
    df['S_intersec/Pob'] = df['S_intersec']/df['Pob']
    df['P_intersec/Sob'] = df['P_intersec']/df['Sob']
    
    for el in ['S_intersec','P_intersec', 'Sob/S_intersec', 'Pob/P_intersec', 'S_intersec/Sob','P_intersec/Pob', 'S_intersec/Pob', 'P_intersec/Sob', 'dd1', 'dd2', 'dd3', 'dd4', 'dd5', 'dd6']:
        df[el+'_log10'] = np.log10(df[el])
    
    gr = df.groupby('userId')['itemId'].nunique().values
    us = sorted(df.userId.unique())
    gr = pd.DataFrame(gr, columns=['count_user_item'])
    us = pd.DataFrame(us, columns=['userId'])
    gr =  pd.concat([us, gr], axis=1, sort=False)
    df =  pd.merge(df, gr, how = 'left',
                      left_on  = ['userId'],
                      right_on = ['userId'])
    gr = df.groupby('itemId')['userId'].nunique().values
    us = sorted(df.itemId.unique())
    gr = pd.DataFrame(gr, columns=['count_item_user'])
    us = pd.DataFrame(us, columns=['itemId'])
    gr =  pd.concat([us, gr], axis=1, sort=False)
    df =  pd.merge(df, gr, how = 'left',
                      left_on  = ['itemId'],
                      right_on = ['itemId'])
    
    
    scan = clustering.fit_predict(df[['Xmin','Ymin', 'Xmax', 'Ymax']].values)
    scan = pd.DataFrame(scan, columns=['dbscan'])
    df = pd.concat([df, scan], axis=1)
    
    df['dx']=df['Xmin']-df['Ymin']
    df['dy']=df['Xmax']-df['Ymax']
    df['xd']=df['Xmin']-df['Ymax']
    df['yd']=df['Xmax']-df['Ymin']
    df['distancexy']=(df['dx']**2+df['dy']**2)**(1/2)
    df['distanceyx']=(df['xd']**2+df['yd']**2)**(1/2)
    df['distancexy_']=(df['dx']**2+df['yd']**2)**(1/2)
    df['distanceyx_']=(df['xd']**2+df['dy']**2)**(1/2)
    
    def get_cl(s):
        if s>=1900:
            return 1
        if s>=1700:
            return 2
        if s>=1500:
            return 3
        if s>=1300:
            return 4
        if s>=1100:
            return 5
        if s>=900:
            return 6
        if s>=700:
            return 7
        if s>=500:
            return 8
        if s>=300:
            return 9
        if s>=100:
            return 10
        else:
            return 0

    for elem in ['Xmin','Ymin', 'Xmax', 'Ymax']:
    
        df[elem+'_cl'] = df[elem].apply(get_cl)
        
    sc = scaler.transform(df[['Xmin','Ymin', 'Xmax', 'Ymax']])
    sc = pd.DataFrame(sc, columns=['Xmin_sc','Ymin_sc', 'Xmax_sc', 'Ymax_sc'])
    df = pd.concat([df, sc], axis=1)
    sc2 = scaler2.transform(df[['Xmin','Ymin', 'Xmax', 'Ymax']])
    sc2 = pd.DataFrame(sc2, columns=['Xmin_sc_m','Ymin_sc_m', 'Xmax_sc_m', 'Ymax_sc_m'])
    df = pd.concat([df, sc2], axis=1)
#     новые добавленные фичи
#     df['dd1dd2'] = np.sqrt((df['dd1']*df['dd1']) + (df['dd2']*df['dd2']))
#     df['dd2dd1'] = np.sqrt((df['dd1']*df['dd1']) - (df['dd2']*df['dd2']))
#     for el in ['Sob/S_intersec', 'Pob/P_intersec', 'S_intersec/Sob','P_intersec/Pob', 'S_intersec/Pob', 'P_intersec/Sob', 'dd1', 'dd2', 'dd3', 'dd4', 'dd5', 'dd6']:
#         df[el+'_log'] = np.log(df[el])
#     df['a/b'] = df['a']/df['b']
#     df['a-b'] = df['a']-df['b']
#     df['dd1_log2'] = np.log2(df['dd1'])
#     df['dd1_log1p'] = np.log1p(df['dd1'])
#     df['dd1_sin'] = np.sin(df['dd1'])
#     df['dd1_cos'] = np.cos(df['dd1'])
#     df['dd1_tan'] = np.tan(df['dd1'])
#     df['dd1_arcsin'] = np.arcsin(df['dd1'])
#     df['dd1_arccos'] = np.arccos(df['dd1'])
#     df['dd1_arctan'] = np.arctan(df['dd1'])
#     df['dd1_degrees'] = np.degrees(df['dd1'])
#     df['dd1_radians'] = np.radians(df['dd1'])
#     df['dd1_deg2rad'] = np.deg2rad(df['dd1'])
#     df['dd1_rad2deg'] = np.rad2deg(df['dd1'])
#     df['dd1_exp'] = np.exp(df['dd1'])
#     df['dd1_expm1'] = np.expm1(df['dd1'])
#     df['dd1_exp2'] = np.exp2(df['dd1'])
#     df['dd1_sqrt'] = np.sqrt(df['dd1'])
#     df['dd1_cbrt'] = np.cbrt(df['dd1'])
#     df['dd1_square'] = np.square(df['dd1'])
#     df['dd2_log2'] = np.log2(df['dd2'])
#     df['dd2_log1p'] = np.log1p(df['dd2'])
#     df['dd2_sin'] = np.sin(df['dd2'])
#     df['dd2_cos'] = np.cos(df['dd2'])
#     df['dd2_tan'] = np.tan(df['dd2'])
#     df['dd2_arcsin'] = np.arcsin(df['dd2'])
#     df['dd2_arccos'] = np.arccos(df['dd2'])
#     df['dd2_arctan'] = np.arctan(df['dd2'])
#     df['dd2_degrees'] = np.degrees(df['dd2'])
#     df['dd2_radians'] = np.radians(df['dd2'])
#     df['dd2_deg2rad'] = np.deg2rad(df['dd2'])
#     df['dd2_rad2deg'] = np.rad2deg(df['dd2'])
#     df['dd2_exp'] = np.exp(df['dd2'])
#     df['dd2_expm1'] = np.expm1(df['dd2'])
#     df['dd2_exp2'] = np.exp2(df['dd2'])
#     df['dd2_sqrt'] = np.sqrt(df['dd2'])
#     df['dd2_cbrt'] = np.cbrt(df['dd2'])
#     df['dd2_square'] = np.square(df['dd2'])
    
    return df

# df_test = make_feature(test)
# df_train = make_feature(train)

In [3]:
first_bad_us = [969, 415, 1677, 1504, 681, 1601, 314, 1065, 138, 698, 1086, 25, 1377, 426, 145, 706, 247, 482, 132, 537] # 0.5680686 Максимум от этого дропа на минмаксе
df_test_for_cat = test.copy()
print (len(df_test_for_cat))
for el in first_bad_us:
    df = df_test_for_cat[df_test_for_cat.userId==el]
    ind = df.index
    try:
        df_test_for_cat = df_test_for_cat.drop(df_test_for_cat.index[[ind]])
    except IndexError:
        continue

print (len(df_test_for_cat))
print(len(df_test_for_cat.itemId.unique()))
print (df_test_for_cat.head())

df_test_for_cat = make_feature(df_test_for_cat)
print (df_test_for_cat.head())



print (len(train))
for el in first_bad_us:
    df = train[train.userId==el]
    ind = df.index
    try:
        train = train.drop(train.index[[ind]])
    except IndexError:
        continue

print (len(train))
print(len(train.itemId.unique()))
print (train.head())
df_train = make_feature(train)
print (df_train.head())

minmax_without_fs_bad = df_test_for_cat.groupby("itemId")[['Xmin_min','Ymin_min', 'Xmax_max', 'Ymax_max']].mean().reset_index()
minmax_without_fs_bad.to_csv("subm_without_fs_bad.csv",header=None,index=False)
minmax_without_fs_bad.head()

3615
3555
630
   userId  itemId  Xmin  Ymin  Xmax  Ymax
0    1581   34804    86   640   242   743
1    1351   34804    85   655   273   766
2     161   34804    85   648   268   748
3     313    5704    32   618   647  1268
4    1097    5704    25   620   602  1242
   userId  itemId  Xmin  Ymin  Xmax  Ymax  Xmin_mean   Ymin_mean   Xmax_mean  \
0    1581   34804    86   640   242   743  85.333333  647.666667  261.000000   
1    1351   34804    85   655   273   766  85.333333  647.666667  261.000000   
2     161   34804    85   648   268   748  85.333333  647.666667  261.000000   
3     313    5704    32   618   647  1268  58.333333  642.666667  610.666667   
4    1097    5704    25   620   602  1242  58.333333  642.666667  610.666667   

     Ymax_mean    ...      Xmax_cl  Ymax_cl   Xmin_sc   Ymin_sc   Xmax_sc  \
0   752.333333    ...           10        7 -0.313448 -0.060028 -0.742633   
1   752.333333    ...           10        7 -0.323047 -0.002202 -0.618583   
2   752.333333    ... 

,itemId,Xmin_min,Ymin_min,Xmax_max,Ymax_max
0,18,126,634,764,1132
1,19,27,559,119,729
2,33,21,380,414,652
3,62,35,824,639,1262
4,114,41,570,216,780


In [4]:
df_train['Xmin_d'] = df_train.Xmin_true-df_train.Xmin_min
df_train['Ymin_d'] = df_train.Ymin_true-df_train.Ymin_min
df_train['Xmax_d'] = df_train.Xmax_true-df_train.Xmax_max
df_train['Ymax_d'] = df_train.Ymax_true-df_train.Ymax_max
df_train.tail()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,...,Xmax_sc,Ymax_sc,Xmin_sc_m,Ymin_sc_m,Xmax_sc_m,Ymax_sc_m,Xmin_d,Ymin_d,Xmax_d,Ymax_d
5233,1521,20735,60,547,387,789,105,564,384,761,...,-0.162399,-0.488271,0.066047,0.339812,0.359691,0.350437,78,30,-6,-33
5234,157,20735,35,538,383,772,105,564,384,761,...,-0.178405,-0.550547,0.040248,0.333780,0.355834,0.340525,78,30,-6,-33
5235,612,28736,72,757,260,881,38,743,285,936,...,-0.670604,-0.151250,0.078431,0.480563,0.237223,0.404082,-21,-9,25,54
5236,278,28736,59,756,256,872,38,743,285,936,...,-0.686611,-0.184220,0.065015,0.479893,0.233365,0.398834,-21,-9,25,54
5237,61,28736,108,752,211,882,38,743,285,936,...,-0.866684,-0.147587,0.115583,0.477212,0.189971,0.404665,-21,-9,25,54


In [5]:
from sklearn.metrics import mean_squared_error

df_train = df_train.replace([np.inf, -np.inf], np.nan)
df_train = df_train.fillna(0)
df_test_for_cat = df_test_for_cat.replace([np.inf, -np.inf], np.nan)
df_test_for_cat = df_test_for_cat.fillna(0)

def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False
def errors(y_true, y_pred):
    return 'MSE', mean_squared_error(y_true, y_pred),rmsle(y_true, y_pred),rae(y_true, y_pred)

columns_test = [el for el in df_test_for_cat.columns if el not in ['userId', 'itemId']]
columns_train = [el for el in df_train.columns if el not in ['userId', 'itemId']]

four_cols = ['Xmin_d','Ymin_d', 'Xmax_d', 'Ymax_d']

X_train, X_valid, Y_train, Y_valid = df_train[columns_train],df_train[columns_train][4049:5337],  df_train[four_cols],df_train[four_cols][4049:5337] 

# X_train_1, X_valid_1, Y_train_1, Y_valid_1 = df_train[columns_train][0:4049],df_train[columns_train][4049:5337],  df_train[four_cols][0:4049],df_train[four_cols][4049:5337] 
# X_train_2, X_valid_2, Y_train_2, Y_valid_2 = df_train[columns_train][1128:5337],df_train[columns_train][0:1128],  df_train[four_cols][1128:5337],df_train[four_cols][0:1128] 
# X_train_3, X_valid_3, Y_train_3, Y_valid_3 = pd.concat([df_train[columns_train][0:1128], df_train[columns_train][2460:5337]], axis=0),df_train[columns_train][1128:2460],  pd.concat([df_train[four_cols][0:1128], df_train[four_cols][2460:5337]], axis=0),df_train[four_cols][1128:2460] 
# X_train_4, X_valid_4, Y_train_4, Y_valid_4 = pd.concat([df_train[columns_train][0:2460], df_train[columns_train][3795:5337]], axis=0),df_train[columns_train][2460:3795],  pd.concat([df_train[four_cols][0:2460], df_train[four_cols][3795:5337]], axis=0),df_train[four_cols][2460:3795] 

X_test = df_test_for_cat[columns_test]

In [6]:
# import xgboost 
# xgb = xgboost.XGBRegressor(n_estimators=300, learning_rate=0.2, gamma=0, subsample=0.75,colsample_bytree=1, max_depth=35)
# X = X_valid.copy()
# Test = X_test.copy()
# for el in ['Xmin_d','Ymin_d', 'Xmax_d', 'Ymax_d']: 
#     xgb.fit(X_train[columns_test].values, Y_train[el].values)
#     pred = xgb.predict(X_valid[columns_test].values)
#     pred2 = xgb.predict(X_test.values)
#     X.index = pd.DataFrame(pred,columns=[el+'_PRED']).index
#     Test.index = pd.DataFrame(pred2,columns=[el+'_PRED']).index
#     X = pd.concat([X, pd.DataFrame(pred,columns=[el+'_PRED'])], axis=1)
#     Test = pd.concat([Test, pd.DataFrame(pred2,columns=[el+'_PRED'])], axis=1)
#     print (el, ':', errors(Y_valid[el], pred))
# df_mod_xgb = X.copy()
# df_pred_xgb = Test.copy()

import lightgbm as lgbm 
params = {
        'objective' :'regression',
        'learning_rate' : 0.02,
        'num_leaves' : 76,
        'feature_fraction': 0.64, 
        'bagging_fraction': 0.8, 
        'bagging_freq':1,
        'boosting_type' : 'gbdt',
        'metric': 'mape'
    }
X = X_valid.copy()
Test = X_test.copy()
for el in ['Xmin_d','Ymin_d', 'Xmax_d', 'Ymax_d']: 
    d_train = lgbm.Dataset(X_train[columns_test], Y_train[el])
    d_valid = lgbm.Dataset(X_valid[columns_test], Y_valid[el])
    bst = lgbm.train(params, d_train, 5000, valid_sets=[d_valid], verbose_eval=50, early_stopping_rounds=100)
    pred = bst.predict(X_valid[columns_test])
    pred2 = bst.predict(X_test)
    X.index = pd.DataFrame(pred,columns=[el+'_PRED']).index
    Test.index = pd.DataFrame(pred2,columns=[el+'_PRED']).index
    X = pd.concat([X, pd.DataFrame(pred,columns=[el+'_PRED'])], axis=1)
    Test = pd.concat([Test, pd.DataFrame(pred2,columns=[el+'_PRED'])], axis=1)
    print (':',el, ':', errors(Y_valid[el], pred))

df_mod_xgb = X.copy()
df_pred_xgb = Test.copy()

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's mape: 1.84175
[100]	valid_0's mape: 1.28961
[150]	valid_0's mape: 0.906938
[200]	valid_0's mape: 0.656567
[250]	valid_0's mape: 0.499927
[300]	valid_0's mape: 0.379348
[350]	valid_0's mape: 0.302583
[400]	valid_0's mape: 0.248783
[450]	valid_0's mape: 0.212965
[500]	valid_0's mape: 0.182019
[550]	valid_0's mape: 0.15691
[600]	valid_0's mape: 0.13825
[650]	valid_0's mape: 0.123962
[700]	valid_0's mape: 0.111103
[750]	valid_0's mape: 0.100208
[800]	valid_0's mape: 0.0905097
[850]	valid_0's mape: 0.0823983
[900]	valid_0's mape: 0.0760485
[950]	valid_0's mape: 0.0693238
[1000]	valid_0's mape: 0.0631674
[1050]	valid_0's mape: 0.0577143
[1100]	valid_0's mape: 0.0535028
[1150]	valid_0's mape: 0.0495262
[1200]	valid_0's mape: 0.0462498
[1250]	valid_0's mape: 0.0436022
[1300]	valid_0's mape: 0.0404777
[1350]	valid_0's mape: 0.0379695
[1400]	valid_0's mape: 0.0355782
[1450]	valid_0's mape: 0.0334944
[1500]	valid_0's ma

In [7]:
result_out2 = pd.concat([df_test_for_cat[['userId', 'itemId','Xmin_min','Ymin_min', 'Xmax_max', 'Ymax_max','Xmin','Ymin', 'Xmax', 'Ymax']], df_pred_xgb[['Xmin_d_PRED','Ymin_d_PRED', 'Xmax_d_PRED', 'Ymax_d_PRED']]], axis=1)
# result_out2 = result_out2.groupby("itemId")[['Xmin_min','Ymin_min', 'Xmax_max', 'Ymax_max', 'Xmin_true_PRED','Ymin_true_PRED','Xmax_true_PRED','Ymax_true_PRED']].mean().reset_index()
# result_out2['Xmin'] = (result_out2.Xmin_min+result_out2.Xmin_d_PRED*0.1)
# result_out2['Ymin'] = (result_out2.Ymin_min+result_out2.Ymin_d_PRED*0.1)
result_out2['Xmax'] = (result_out2.Xmax_max+result_out2.Xmax_d_PRED*0.1)
# result_out2['Ymax'] = (result_out2.Ymax_max+result_out2.Ymax_d_PRED*0.1)
result_out_avg = result_out2.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].mean().reset_index()
result_out_avg.to_csv("subm_avg2.csv",header=None,index=False)
result_out_avg.head()

,itemId,Xmin,Ymin,Xmax,Ymax
0,18,135.000000,704.000000,765.431440,1071.000000
1,19,29.666667,560.666667,120.837484,725.666667
2,33,46.333333,393.666667,416.100274,633.666667
3,62,90.857143,964.857143,636.573808,1167.571429
4,114,63.333333,575.000000,217.110458,767.000000


In [8]:
result_out2

,userId,itemId,Xmin_min,Ymin_min,Xmax_max,Ymax_max,Xmin,Ymin,Xmax,Ymax,Xmin_d_PRED,Ymin_d_PRED,Xmax_d_PRED,Ymax_d_PRED
0,1581,34804,85,640,273,766,86,640,272.237748,743,-25.628854,0.938918,-7.622520,-6.736999
1,1351,34804,85,640,273,766,85,655,272.066333,766,-26.184518,-0.312322,-9.336667,-4.148541
2,161,34804,85,640,273,766,85,648,272.028262,748,-25.675426,-1.046080,-9.717381,-4.276293
3,313,5704,25,618,647,1268,32,618,653.859911,1268,41.051662,56.838766,68.599113,-171.069309
4,1097,5704,25,618,647,1268,25,620,654.110535,1242,45.169584,57.247338,71.105346,-171.267398
5,1525,5704,25,618,647,1268,118,690,654.536356,1191,43.206066,53.781918,75.363563,-162.645427
6,957,32082,40,414,262,588,50,437,265.050220,575,-8.477744,-4.698497,30.502199,0.742843
7,947,32082,40,414,262,588,42,414,265.206728,583,-7.741999,1.616209,32.067276,1.622041
8,237,32082,40,414,262,588,63,424,265.459863,569,-9.402132,-2.189147,34.598626,2.522635
9,191,32082,40,414,262,588,59,483,265.215058,584,-6.983471,-2.325526,32.150577,2.674611
